<a href="https://colab.research.google.com/github/madhuameneni/Hand_guesture_rating_DeepLearning_Model/blob/main/hand_guesture_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
!pip install -q keras

In [4]:
import keras

In [5]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
# more info on callbakcs: https://keras.io/callbacks/ model saver is cool too.
from tensorflow.keras.callbacks import TensorBoard
import pickle
import time

In [6]:
NAME = "hand_guestures-CNN"
import numpy as np

In [7]:
pickle_in = open("/content/drive/MyDrive/data/X.pickle","rb")
X = pickle.load(pickle_in)

pickle_in = open("/content/drive/MyDrive/data/y.pickle","rb")
y = pickle.load(pickle_in)

X = X/255.0
y = np.array(y)

In [13]:
# Create a CNN Sequential Model
model = Sequential()
model.add(Conv2D(32, (5,5), activation = 'relu', input_shape=X.shape[1:]))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(64, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu')) 
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dropout(0.3))
model.add(Dense(128, activation='relu'))
model.add(Dense(3, activation='softmax'))
#Model configuration for training purpose
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model.fit(X, y,
                      batch_size=32,
                      epochs=10,
                      validation_split=0.3,
                      callbacks=[tensorboard])
model.save('64x3-CNN.model')


Epoch 1/10
92/92 [==============================] - 2s 16ms/step - loss: 0.7035 - accuracy: 0.6379 - val_loss: 0.1846 - val_accuracy: 0.9421
Epoch 2/10
92/92 [==============================] - 1s 13ms/step - loss: 0.1442 - accuracy: 0.9469 - val_loss: 0.0628 - val_accuracy: 0.9794
Epoch 3/10
92/92 [==============================] - 1s 13ms/step - loss: 0.0618 - accuracy: 0.9799 - val_loss: 0.0587 - val_accuracy: 0.9833
Epoch 4/10
92/92 [==============================] - 1s 13ms/step - loss: 0.0358 - accuracy: 0.9864 - val_loss: 0.0443 - val_accuracy: 0.9905
Epoch 5/10
92/92 [==============================] - 1s 13ms/step - loss: 0.0174 - accuracy: 0.9943 - val_loss: 0.0314 - val_accuracy: 0.9944
Epoch 6/10
92/92 [==============================] - 1s 13ms/step - loss: 0.0222 - accuracy: 0.9945 - val_loss: 0.0514 - val_accuracy: 0.9881
Epoch 7/10
92/92 [==============================] - 1s 13ms/step - loss: 0.0187 - accuracy: 0.9930 - val_loss: 0.0475 - val_accuracy: 0.9952
Epoch 8/10
92

In [22]:
import cv2
import tensorflow as tf

CATEGORIES = ["one", "two", "three"]
def prepare(filepath):
    IMG_SIZE = 100
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)


model = tf.keras.models.load_model("64x3-CNN.model")

prediction = model.predict([prepare('/content/drive/MyDrive/data/three.jpg')])
print(prediction)  # will be a list in a list.
print(CATEGORIES[int(prediction[0][0])])
y_pred = np.argmax(prediction, axis=1)
print(y_pred)

[[0. 0. 1.]]
one
[2]


In [23]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_float_model = converter.convert()

# Show model size in KBs.
float_model_size = len(tflite_float_model) / 1024
print('Float model size = %dKBs.' % float_model_size)

INFO:tensorflow:Assets written to: /tmp/tmpk_ev15kx/assets
Float model size = 1970KBs.


In [24]:
# Re-convert the model to TF Lite using quantization.
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quantized_model = converter.convert()

# Show model size in KBs.
quantized_model_size = len(tflite_quantized_model) / 1024
print('Quantized model size = %dKBs,' % quantized_model_size)
print('which is about %d%% of the float model size.'\
      % (quantized_model_size * 100 / float_model_size))

INFO:tensorflow:Assets written to: /tmp/tmp6dwcsxt3/assets


INFO:tensorflow:Assets written to: /tmp/tmp6dwcsxt3/assets


Quantized model size = 507KBs,
which is about 25% of the float model size.


In [38]:
def prepare_test(filepath):
    IMG_SIZE = 224
    img_array = cv2.imread(filepath, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

def normalize(row):
    image, label = row['image'], row['label']
    image = tf.dtypes.cast(image, tf.float32)
    image = tf.image.resize(image, (224, 224))
    image = image / 255.0
    return image, label

def evaluate_tflite_model(tflite_model):
  # Initialize TFLite interpreter using the model.
  interpreter = tf.lite.Interpreter(model_content=tflite_model)
  interpreter.allocate_tensors()
  input_tensor_index = interpreter.get_input_details()[0]["index"]
  output = interpreter.tensor(interpreter.get_output_details()[0]["index"])
  test_image = image = tf.dtypes.cast(prepare('/content/drive/MyDrive/data/one.jpg'), tf.float32)
  interpreter.set_tensor(input_tensor_index, test_image)

    # Run inference.
  interpreter.invoke()

    # Post-processing: remove batch dimension and find the digit with highest
    # probability.
  digit = np.argmax(output()[0])
  print(digit)

evaluate_tflite_model(tflite_quantized_model)


0


In [40]:
# Save the quantized model to file to the Downloads directory
f = open('hand_rating.tflite', "wb")
f.write(tflite_quantized_model)
f.close()

# Download the digit classification model
from google.colab import files
files.download('hand_rating.tflite')

print('`mnist.tflite` has been downloaded')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

`mnist.tflite` has been downloaded
